<a href="https://colab.research.google.com/github/SwaksharDeb/Activity-recognition/blob/master/HAR_paper1_v2_mem_optimized.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Importing dependecies

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
import csv
import array
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from numpy import savetxt
from sklearn.preprocessing import StandardScaler
from collections import Counter
from sklearn.model_selection import train_test_split
import tensorflow as tf
import scipy.stats as stats
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from math import sqrt
import math

# for garbage collection
import gc

# Import dataset

In [ ]:
#@title Load datasets from folder
#@markdown Outputs a list of all csv files in folder into <i> data_list </i>

folder_path = "/content/drive/My Drive/nurse care data/swakshar" #@param {type : "string"}

import os

data_files = os.listdir(folder_path)
data_list = []

for data_file in data_files:
    full_file_name = os.path.join(folder_path, data_file)

    data_list.append(pd.read_csv(full_file_name))
    print(f"{full_file_name} read and appended.")

/content/drive/My Drive/nurse care data/swakshar/final_field_user04_dataset.csv read and appended.
/content/drive/My Drive/nurse care data/swakshar/lab data localized.csv read and appended.
/content/drive/My Drive/nurse care data/swakshar/final_field_user51_dataset.csv read and appended.
/content/drive/My Drive/nurse care data/swakshar/final_field_user07_dataset.csv read and appended.
/content/drive/My Drive/nurse care data/swakshar/final_field_user38_dataset.csv read and appended.
/content/drive/My Drive/nurse care data/swakshar/final_field_user18_dataset.csv read and appended.
/content/drive/My Drive/nurse care data/swakshar/final_field_user08_dataset.csv read and appended.


In [ ]:
dataset = pd.concat(data_list, ignore_index=True)

In [ ]:
del data_list

In [ ]:
dataset

,x,y,z,act_id
0,-2.221,7.776,6.366,5.0
1,1.455,7.125,5.890,5.0
2,1.455,7.125,5.890,5.0
3,0.574,6.205,6.653,5.0
4,2.260,5.171,7.721,5.0
...,...,...,...,...
1867772,3.447,6.818,4.217,12.0
1867773,3.447,6.818,4.217,12.0
1867774,-0.306,6.857,6.618,12.0
1867775,-0.306,6.857,6.618,12.0


# Data processing

In [ ]:
""" Data processing"""
X = dataset.iloc[:, [0, 1, 2]].values
Y = dataset.iloc[:, 3].values
Y = Y - 1

## Data Augmentation

we just use scaling for data augmentation. The minority classes are scaled and added to the original data to make the original data balance. Our assumption is that, if we scale a signal it still represent the same activity

In [ ]:
# Data Augmentation
scalling_factor = [0.3,0.5,0.7,0.8,1.2,1.5,2]

for scal in scalling_factor:
    X_1_scal = X[np.where(Y==1)]*scal
    label_1 = Y[np.where(Y==1)]
    X_2_scal = X[np.where(Y==2)]*scal
    label_2 = Y[np.where(Y==2)]
    X_5_scal = X[np.where(Y==5)]*scal
    label_5 = Y[np.where(Y==5)]
    X_7_scal = X[np.where(Y==7)]*scal
    label_7 = Y[np.where(Y==7)]
    X_8_scal = X[np.where(Y==8)]*scal
    label_8 = Y[np.where(Y==8)]
    X_9_scal = X[np.where(Y==9)]*scal
    label_9 = Y[np.where(Y==9)]
    X_10_scal = X[np.where(Y==10)]*scal
    label_10 = Y[np.where(Y==10)]
    X = np.concatenate((X, X_1_scal,X_2_scal,X_5_scal,X_7_scal,X_8_scal,X_9_scal,X_10_scal))
    Y = np.concatenate((Y, label_1,label_2, label_5, label_7, label_8, label_9, label_10))

scalling_factor = [0.5,2]

for scal in scalling_factor:
    X_3_scal = X[np.where(Y==3)]*scal
    label_3 = Y[np.where(Y==3)]
    X_11_scal = X[np.where(Y==11)]*scal
    label_11 = Y[np.where(Y==11)]
    X = np.concatenate((X, X_3_scal, X_11_scal))
    Y = np.concatenate((Y, label_3, label_11))

scalling_factor = [0.3,0.5,1.5,2]

for scal in scalling_factor:
    X_0_scal = X[np.where(Y==0)]*scal
    label_0 = Y[np.where(Y==0)]
    X_6_scal = X[np.where(Y==6)]*scal
    label_6 = Y[np.where(Y==6)]
    X = np.concatenate((X, X_0_scal, X_6_scal))
    Y = np.concatenate((Y, label_0, label_6))

In [ ]:
garbage = [X_0_scal, X_1_scal, X_2_scal, X_3_scal, X_5_scal, X_6_scal, X_7_scal, X_8_scal, X_9_scal, X_10_scal, X_11_scal, label_0, label_1, label_2, label_3, label_5, label_6, label_7, label_8, label_9, label_10, label_11]

del garbage

In [ ]:
gc.collect()

33

## Low pass filter

In [ ]:
def low_pass_filter(input):
    low_pass_data = np.zeros((input.shape))

    for i in range(len(input)):
        for j in range(input.shape[1]):
            if i == 0:
                low_pass_data[i][j] = input[i][j]
            else:
                low_pass_data[i][j] = 0.8*input[i-1][j] + (1-0.8)*input[i][j]

    return low_pass_data


## Frame preparation

In [ ]:
Fs = 4
POLLING_RATE = 4 #Hz
FRAME_SIZE = Fs*20 # 80 # 20 seconds of data
HOP_SIZE =  Fs*10

def get_frames(df, label_data):
    N_FEATURES = 3

    frames = []
    labels = []
    for i in range(0, len(df) - FRAME_SIZE, HOP_SIZE):
        value = df[i:i + FRAME_SIZE, :]

        # Retrieve the most often used label in this segment
        label = stats.mode(label_data[i: i + FRAME_SIZE])[0][0]
        frames.append([value])
        labels.append(label)

    # Bring the segments into a better shape
    frames = np.asarray(frames).reshape(-1, FRAME_SIZE, N_FEATURES)
    labels = np.asarray(labels)

    return frames, labels


## Feature calculation

### Define all feature calculation functions

In [ ]:
def calculate_mean(low_pass_frame):
    return np.mean(low_pass_frame, axis=1)

def calculate_std(low_pass_frame):
    return np.std(low_pass_frame, axis=1)

def calculate_rms(low_pass_frame):
    rms_x = []
    rms_y = []
    rms_z = []

    values = [0, 0, 0]
    no_frames, frame_length, no_axes = low_pass_frame.shape
    for i in range(no_frames):
        for j in range(frame_length):
            for k in range(no_axes):
                values[k] += (low_pass_frame[i][j][k] ** 2)

        values = [sqrt(value / frame_length) for value in values]

        rms_x.append(values[0])
        rms_y.append(values[1])
        rms_z.append(values[2])

        values = [0, 0, 0]

    return rms_x, rms_y, rms_z

def calculate_vec_length(frame):
    lengths = []

    nx, ny, nz = frame.shape
    acc_vector = np.reshape(frame, (nx, ny * nz))

    for i in range(acc_vector.shape[0]):
        length = 0
        count = 0

        for j in range(acc_vector.shape[1]):
            length += acc_vector[i][j] ** 2
            count += 1

            if count == 3:
                lengths.append(sqrt(length))
                length = 0
                count = 0

    return lengths

def calculate_avc(lengths):
    avc = []

    for i in range(0, len(lengths), FRAME_SIZE):
        value = 0
        iterable = lengths[i:i+FRAME_SIZE]
        for j in range(len(iterable) - 1, 1, -1):
            value += abs(iterable[j] - iterable[j - 1])

        value /= (FRAME_SIZE / POLLING_RATE)
        avc.append(value)

    return avc

def calculate_max_min(lengths):
    max_min = []
    for i in range(0, len(lengths), FRAME_SIZE):
        iterable = lengths[i:i+FRAME_SIZE]

        max_min.append(max(iterable) - min(iterable))

    return max_min

def calculate_angles(low_pass_frame):
    angle = np.zeros(low_pass_frame.shape)

    no_frames, frame_length, no_axes = angle.shape
    length = 0
    for i in range(no_frames):
        for j in range(frame_length):
            for k in range(no_axes):
                length += low_pass_frame[i, j, k] ** 2
            length = sqrt(length)

            if length == 0:
                print("length is zero in index row= ", i, "column= ", j)
            else:
                angle_x = math.acos(low_pass_frame[i, j, 0] / length)
                angle_y = math.acos(low_pass_frame[i, j, 1] / length)
                angle_z = math.acos(low_pass_frame[i, j, 2] / length)
                angle[i][j][0] = angle_x
                angle[i][j][1] = angle_y
                angle[i][j][2] = angle_z

    nx, ny, nz = angle.shape

    return np.reshape(angle, (nx, ny * nz))

### Define input vector calculating function

In [ ]:
def get_input_vector(input_X, input_Y):
    low_pass_X = low_pass_filter(input_X)

    low_pass_frame, low_pass_labels = get_frames(low_pass_X, input_Y)

    angles = calculate_angles(low_pass_frame)
    print("angles done.")
    mean = calculate_mean(low_pass_frame)
    print("mean done.")
    std = calculate_std(low_pass_frame)
    print("std done.")
    rms_x, rms_y, rms_z = calculate_rms(low_pass_frame)
    print("rms done.")
    garbage = [low_pass_X, low_pass_frame, low_pass_labels]
    del garbage
    gc.collect()

    row_frame, row_labels = get_frames(input_X, input_Y)

    lengths = calculate_vec_length(row_frame)
    print("lengths done.")
    avc = calculate_avc(lengths)
    print("avs done.")
    max_min = calculate_max_min(lengths)
    print("maxmin done.")

    input_vector_shape = (row_frame.shape[0], angles.shape[1] + 11)

    garbage = [lengths, row_frame]
    del garbage
    gc.collect()

    input_vector = np.zeros(input_vector_shape)
    for i in range(input_vector.shape[0]):
        input_vector[i][0] = mean[i][0]
        input_vector[i][1] = mean[i][1]
        input_vector[i][2] = mean[i][2]
        input_vector[i][3] = std[i][0]
        input_vector[i][4] = std[i][1]
        input_vector[i][5] = std[i][2]
        input_vector[i][6] = rms_x[i]
        input_vector[i][7] = rms_y[i]
        input_vector[i][8] = rms_z[i]
        input_vector[i][9] = max_min[i]
        input_vector[i][10] = avc[i]

    for i in range(angles.shape[0]):
        for j in range(angles.shape[1]):
            input_vector[i][j+11] = angles[i][j]

    return input_vector, row_labels

In [ ]:
input_vector, final_labels = get_input_vector(X, Y)

angles done.
mean done.
std done.
rms done.
lengths done.
avs done.
maxmin done.


# Standarization

In [ ]:
sc = StandardScaler()
input_vector = sc.fit_transform(input_vector)

# Train Test split

In [ ]:
# X_train, X_val, y_train, y_val = train_test_split(input_vector, final_labels, test_size=0.2, stratify=final_labels)

In [ ]:
og_X = dataset.iloc[:, [0, 1, 2]].values
og_Y = (dataset.iloc[:, 3].values) - 1

X_val, y_val = get_input_vector(og_X, og_Y)
_, X_val, _, y_val = train_test_split(X_val, y_val, test_size=0.2, stratify=y_val)

# length = round(len(og_X) * 0.2)
# start_point = np.random.randint(0, len(og_X) - length)
# end_point = start_point + length + 1

# og_X = og_X[start_point:end_point]
# og_Y = og_Y[start_point:end_point]

X_train, y_train = input_vector, final_labels

angles done.
mean done.
std done.
rms done.
lengths done.
avs done.
maxmin done.


In [ ]:
X_val = sc.fit_transform(X_val)

In [ ]:
Counter(y_val)

Counter({0.0: 138,
         1.0: 51,
         2.0: 527,
         3.0: 2088,
         4.0: 5410,
         5.0: 112,
         6.0: 581,
         7.0: 31,
         8.0: 45,
         9.0: 43,
         10.0: 39,
         11.0: 274})

# Algorithm

## Neural Network

In [ ]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_shape=(X_train.shape[1], )))
#model.add(Dropout(0.2))
model.add(Dense(units=64, activation='relu'))
#model.add(Dropout(0.2))
model.add(Dense(units=128, activation='relu'))
#model.add(Dropout(0.2))
#model.add(Dense(units=324, activation='relu'))
#model.add(Dropout(0.2))
model.add(Dense(units=12, activation='softmax'))

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_val, y_val), verbose=1)

Epoch 1/50
10544/10544 [==============================] - 17s 2ms/step - loss: 1.4980 - accuracy: 0.5781 - val_loss: 3.3130 - val_accuracy: 0.0817
Epoch 2/50
10544/10544 [==============================] - 18s 2ms/step - loss: 1.3425 - accuracy: 0.6017 - val_loss: 3.4682 - val_accuracy: 0.0791
Epoch 3/50
10544/10544 [==============================] - 17s 2ms/step - loss: 1.2867 - accuracy: 0.6096 - val_loss: 3.7662 - val_accuracy: 0.0797
Epoch 4/50
10544/10544 [==============================] - 17s 2ms/step - loss: 1.2499 - accuracy: 0.6172 - val_loss: 3.8414 - val_accuracy: 0.0866
Epoch 5/50
10544/10544 [==============================] - 18s 2ms/step - loss: 1.2211 - accuracy: 0.6230 - val_loss: 4.1237 - val_accuracy: 0.0853
Epoch 6/50
10544/10544 [==============================] - 18s 2ms/step - loss: 1.1973 - accuracy: 0.6275 - val_loss: 4.0102 - val_accuracy: 0.0819
Epoch 7/50
10544/10544 [==============================] - 17s 2ms/step - loss: 1.1768 - accuracy: 0.6314 - val_loss: 4

In [ ]:
y_pred = model.predict(X_val)

In [ ]:
y_pred_ = np.zeros((y_pred.shape[0]))
for i in range(y_pred.shape[0]):
    y_pred_[i] = np.argmax(y_pred[i,:])

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_pred_, y_val)

In [ ]:
pd.DataFrame(cm)

,0,1,2,3,4,5,6,7,8,9,10,11
0,5,2,2,12,35,1,5,0,0,1,0,4
1,6,12,2,23,24,5,5,2,3,2,2,6
2,71,20,437,741,3837,29,246,20,31,25,17,142
3,15,2,31,280,259,8,42,2,2,2,3,7
4,0,0,2,3,18,0,4,0,0,0,0,0
5,21,3,26,632,653,37,56,2,5,6,9,30
6,10,1,7,331,521,21,208,0,0,1,1,70
7,0,1,0,5,0,1,0,0,0,0,1,2
8,7,5,11,25,24,0,5,0,2,3,1,6
9,2,2,7,17,27,8,6,0,0,1,1,3


In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_val, y_pred_)

In [ ]:
per_class_accuracy = {}
for i in range(cm.shape[0]):
    accuracy = cm[i,i]/sum(cm[i,:])
    per_class_accuracy[i] = accuracy

In [ ]:
per_class_accuracy

{0: 0.07462686567164178,
 1: 0.13043478260869565,
 2: 0.07781339031339031,
 3: 0.42879019908116384,
 4: 0.6666666666666666,
 5: 0.025,
 6: 0.1776259607173356,
 7: 0.0,
 8: 0.02247191011235955,
 9: 0.013513513513513514,
 10: 0.06896551724137931,
 11: 1.0}